In [ ]:
!git clone https://github.com/arcee-ai/DistillKit.git

In [ ]:
!./DistillKit/setup.sh

In [ ]:
!pip install trl==0.9.6 accelerate==1.0.1 wandb

In [ ]:
from huggingface_hub import snapshot_download

# Replace "model-repo-id" with your desired model repo (e.g., "stabilityai/stable-diffusion-2-1")
snapshot_download(repo_id="grounded-ai/phi4-r1-guard")
snapshot_download(repo_id="microsoft/Phi-3.5-mini-instruct")

In [13]:
!accelerate launch ./DistillKit/distill_logits.py --mixed_precision=='yes'

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
[2025-02-19 23:20:19,513] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Preprocessing and tokenizing dataset...
Map (num_proc=8): 100%|██████████████| 522/522 [00:00<00:00, 1829.29 examples/s]
Dataset preparation complete. Loading models...
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:00<00:00,  6.81it/s]
Spectrum configuration not found. All layers of the student model will be t

![WANDB](dist.png)

In [99]:
# test
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
tokenizer.padding_side = 'right'

def run_merged_model(messages):
  # input = format_func(query, text)
  # messages = [
  #     {"role": "user", "content": input}
  # ]

  pipe = pipeline(
      "text-generation",
      model='./results',
      tokenizer=tokenizer,
      device='cuda'
  )

  generation_args = {
      "max_new_tokens": 756,
      "return_full_text": False,
      "temperature": 0.7,
      "do_sample": True,
  }

  output = pipe(messages, **generation_args)
  torch.cuda.empty_cache()
  return output[0]['generated_text'].strip().lower()

In [45]:
from datasets import load_from_disk

ds = load_from_disk('distil_data')

# Example 1: Hallucinations

In [100]:
record = ds.to_pandas().iloc[20].prompt
record = record[0:2]
record = record.tolist()
record

[{'content': 'Your job is to evaluate whether a machine learning model has hallucinated or not. A hallucination occurs when the response is coherent but factually incorrect or nonsensical outputs that are not grounded in the provided context. \n\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n',
  'role': 'system'},
 {'content': 'You are given the following information:\n                    <info>\n                    [Knowledge]: "Alive" is a song by The Black Eyed Peas from their album "The E.N.D". It was the second promotional single from the album as part of "The Countdown to "The E.N.D", the first being "Imma Be" and the third being "Meet Me Halfway" which were later commercial singles."Meet Me Halfway" is the third single from the Black Eyed Peas’ fifth studio album "The E.N.D".\n                    [User Input]: What band wrote the songs Alive and Meet Me Halfway were the first and third songs on their album called "The E.N.D.?\n     

In [101]:
answer = ds.to_pandas().iloc[20].answer
answer

'yes'

In [102]:
student_answer = run_merged_model(record)
student_answer

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


'<reasoning>\n the user provided explicit information that "alive" and "meet me halfway" are songs by the black eyed peas from their album "the e.n.d.\n the model response incorrectly states that the black eyed peas did not write "alive" and "meet me halfway". this is a factual inaccuracy and does not align with the given information.\n</reasoning>\n<answer>\nyes\n</answer>'

In [97]:
teacher_answer = ds.to_pandas().iloc[20].prompt[2]
teacher_answer['content']

'<reasoning>\nThe task is to determine if the model\'s response is a hallucination, which means checking if the response is coherent but factually incorrect or nonsensical based on the provided context.\n\n1. **Context Analysis**: \n   - The provided information states that "Alive" and "Meet Me Halfway" are songs by The Black Eyed Peas from their album "The E.N.D."\n   - It also mentions that "Meet Me Halfway" is the third single from the album.\n\n2. **Model Response**: \n   - The model states, "The Black Eyed Peas didn\'t write \'Alive\' and \'Meet Me Halfway\'."\n\n3. **Fact-Checking**:\n   - According to the provided context, The Black Eyed Peas did write "Alive" and "Meet Me Halfway" as they are songs from their album "The E.N.D."\n   - The model\'s statement contradicts this information.\n\n4. **Conclusion**:\n   - The model\'s response is factually incorrect based on the provided context.\n   - The response is coherent but not grounded in the given information, making it a hallu

# Example 2: RAG Relevance

In [103]:
record = ds.to_pandas().iloc[27].prompt
record = record[0:2]
record = record.tolist()
record

[{'content': 'Your job is to evaluate whether a retrieved context is relevant, or unrelated to a user query. \n\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n',
  'role': 'system'},
 {'content': '\n                You are comparing a reference text to a question and trying to determine if the reference text\n                contains information relevant to answering the question. Here is the info:\n                <info>\n                ************\n                [Question]: what is hosting a website\n                ************\n                [Reference text]: An example of rack mounted servers A web hosting service is a type of Internet hosting service that allows individuals and organizations to make their website accessible via the World Wide Web . Web hosts are companies that provide space on a server owned or leased for use by clients, as well as providing Internet connectivity, typically in a data center . Web hosts can also 

In [104]:
answer = ds.to_pandas().iloc[27].answer
answer

'relevant'

In [105]:
student_answer = run_merged_model(record)
student_answer

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


'<reasoning>\nthe reference text discusses web hosting services, including what they provide, different types of hosting, and some features like ssl for e-commerce. it also mentions that web hosts provide space on a server for clients and internet connectivity, which is the fundamental idea behind hosting a website. the reference text directly addresses the question of what hosting a website is, providing detailed information about the services and features involved. this indicates that the reference text contains relevant information that can answer the question.\n</reasoning>\n<answer>\nrelevant'

In [78]:
teacher_answer = ds.to_pandas().iloc[27].prompt[2]
teacher_answer['content']

'<reasoning>\nThe user query is "what is hosting a website." The reference text provides a detailed explanation of what a web hosting service is, including the types of services offered, the technology involved, and the different levels of hosting available. It explains that web hosting services allow individuals and organizations to make their websites accessible via the World Wide Web by providing server space and Internet connectivity. This directly addresses the user\'s question by defining and describing the concept of hosting a website. Therefore, the reference text contains relevant information that can answer the question.\n</reasoning>\n<answer>\nrelevant\n</answer>'

# Example 3: Toxicity

In [106]:
record = ds.to_pandas().iloc[34].prompt
record = record[0:2]
record = record.tolist()
record

[{'content': 'Your job is to evaluate whether a human text is toxic or non-toxic. \n\nRespond in the following format:\n<reasoning>\n...\n</reasoning>\n<answer>\n...\n</answer>\n',
  'role': 'system'},
 {'content': '\n                You are examining written text content. Here is the text:\n                <info>\n                [Text]: I am not evading, Musarabbyahamd is not my ID. Strange!\n                </info>\n                Examine the text and determine  whether the text is toxic or not. Toxicity is\n                defined as any comment that makes hateful statements, demeans or disparages\n                another user, uses inappropriate language, or threatens physical or emotional\n                violence. Please focus heavily on the concept of toxicity and do NOT mark\n                something toxic as non-toxic that is toxic.\n                "toxic" means that the text meets the definition of toxic. "non-toxic" means the text does not contain any\n                wo

In [107]:
answer = ds.to_pandas().iloc[34].answer
answer

'non-toxic'

In [108]:
student_answer = run_merged_model(record)
student_answer

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


'<reasoning>\nthe given text is a statement from a user stating that their id is not musrabahamd and finds it strange. this statement does not contain any hateful statements, disparaging remarks, inappropriate language, or threats of violence. the user is simply expressing their opinion and providing information about their identity. there is no indication of negativity or harmful intentions in the text.\n\n</reasoning>\n<answer}\nnon-toxic\n</answer>'

In [109]:
teacher_answer = ds.to_pandas().iloc[34].prompt[2]
teacher_answer['content']

'<reasoning>\nThe text provided is: "I am not evading, Musarabbyahamd is not my ID. Strange!" To determine if this text is toxic, we need to assess whether it contains any elements that are defined as toxic: hateful statements, demeaning or disparaging comments, inappropriate language, or threats of physical or emotional violence.\n\n1. **Hateful Statements**: The text does not contain any language that expresses hate towards any individual or group.\n\n2. **Demeaning or Disparaging Comments**: The text does not demean or disparage any user. It simply states that "Musarabbyahamd is not my ID," which is a factual statement about the user\'s identity.\n\n3. **Inappropriate Language**: There is no use of inappropriate language in the text. The language used is straightforward and neutral.\n\n4. **Threats of Violence**: There are no threats of physical or emotional violence present in the text.\n\nThe word "Strange!" at the end of the text might suggest surprise or confusion, but it does n

In [111]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer from the saved directory
model_path = "./results"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Push model and tokenizer to the Hugging Face Hub
model.push_to_hub("Jlonge4/phi4-phi3.5-distil")
tokenizer.push_to_hub("Jlonge4/phi4-phi3.5-distil")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:833: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/394M [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:833: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jlonge4/phi4-phi3.5-distil/commit/a3f7dd55362a11ac8c1104ba80ad767b428db785', commit_message='Upload tokenizer', commit_description='', oid='a3f7dd55362a11ac8c1104ba80ad767b428db785', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Jlonge4/phi4-phi3.5-distil', endpoint='https://huggingface.co', repo_type='model', repo_id='Jlonge4/phi4-phi3.5-distil'), pr_revision=None, pr_num=None)